In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from utils import *
from implementations import *
%load_ext autoreload
%autoreload 2

## Load data

In [2]:
y, X, ids = load_csv_data(path="../data/train.csv")

print("Input data shape", X.shape)
print("Labels data shape", y.shape)

Input data shape (250000, 30)
Labels data shape (250000,)


In [3]:
y

array([ 1., -1., -1., ...,  1., -1., -1.])

In [4]:
y[np.where(y == -1)] = 0
y

array([1., 0., 0., ..., 1., 0., 0.])

In [5]:
x_tr, x_val, y_tr, y_val = split_data(X, y, 0.8)

print("Training data shapes", x_tr.shape, y_tr.shape)
print("Validation data shapes", x_val.shape, y_val.shape)

Training data shapes (200000, 30) (200000,)
Validation data shapes (50000, 30) (50000,)


In [6]:
x_tr, _, _ = standardize(x_tr)
x_tr.shape

(200000, 30)

In [8]:
weights, loss = logistic_regression(y_tr, x_tr, np.random.random(x_tr.shape[1]), max_iters=1000, gamma=0.01)

In [9]:
loss

0.6140079801359767

In [10]:
y_pred = np.array([predictions(x, weights) for x in x_tr])
(y_tr == y_pred).sum() / y_tr.shape[0]

0.66234

In [11]:
x_val, _, _ = standardize(x_val)
x_val.shape

(50000, 30)

In [12]:
y_pred = np.array([predictions(x, weights) for x in x_val])
y_pred

array([0, 0, 1, ..., 0, 0, 0])

In [13]:
(y_val == y_pred).sum() / y_val.shape[0]

0.6594

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(x_tr, y_tr)

C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [8]:
clf.score(x_tr, y_tr)

0.73724

In [9]:
clf.score(x_val, y_val)

0.73722